### Yake

In [3]:
import pke

def yake_extract(text):
    extractor = pke.unsupervised.YAKE()
    extractor.load_document(input=text, language='en')
    extractor.candidate_selection()
    extractor.candidate_weighting()
    keyphrases = extractor.get_n_best()
    returned_keyphrases = []
    for keyphrase in keyphrases:
        keyphrase = keyphrase[0]
        returned_keyphrases.append(keyphrase)
    return ' '.join(returned_keyphrases)

In [ ]:
import pandas as pd
import os
from tqdm import tqdm

project = 'spark'
method = 'yake'

df = pd.read_csv('../data/raw/test_{}.csv'.format(project))
yake_folder = '../data/keywords/{}/yake-idf/run_1/'
flag_content_df = pd.read_csv(f'../data/ablation/test_{project}_flag_content.csv')

if not os.path.exists(yake_folder.format(project)):
    os.makedirs(yake_folder.format(project))

for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    bug_id = row['bug_id']
    if flag_content_df[flag_content_df['bug_id'] == bug_id]['run_flag'].values[0] == 0:
        continue
    if os.path.exists(os.path.join(yake_folder, f'{bug_id}.txt')):
        continue
    short_desc = row['short_desc']
    description = row['description']
    short_desc_keywords = yake_extract(short_desc)
    description_keywords = yake_extract(description)
    
    with open(yake_folder.format(project) + '/{}.txt'.format(bug_id), 'w') as f:
        f.write('Summary: {}'.format(short_desc_keywords))
        f.write('\n')
        f.write('Description: {}'.format(description_keywords))
        f.write('\n')  